In [ ]:
#
# This script systematically trains the Regression Baseline model for all 21
# variable combinations to conduct a scientifically rigorous ablation study.
# It uses the same AdvancedLoss as the main regression model for fair comparison.
#
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import numpy as np
from pathlib import Path
import joblib
import warnings
from tqdm import tqdm
import json
import gc

warnings.filterwarnings('ignore')

# --- 1. CONFIGURATION ---
PROJECT_PATH = Path('/content/drive/My Drive/AR_Downscaling')
DATA_DIR = PROJECT_PATH / 'final_dataset_multi_variable'
OUTPUT_DIR = PROJECT_PATH / 'ablation_study_models'
OUTPUT_DIR.mkdir(exist_ok=True)

# --- Training Hyperparameters (MUST remain constant for all models) ---
EPOCHS = 25
BATCH_SIZE = 8
LEARNING_RATE = 1e-4
WARMUP_EPOCHS = 3
EARLY_STOPPING_PATIENCE = 7
GRADIENT_CLIP = 1.0

# --- 2. ABLATION STUDY CONFIGURATION ---
ALL_VARIABLES = ['IVT', 'T500', 'T850', 'RH700', 'W500']

ABLATION_CONFIGS = {
    # --- Single Variable Models (5 total) ---
    'single_IVT': ['IVT'],
    'single_T500': ['T500'],
    'single_T850': ['T850'],
    'single_RH700': ['RH700'],
    'single_W500': ['W500'],
    # --- Two Variable Models (6 total from your script) ---
    'pair_IVT_T500': ['IVT', 'T500'],
    'pair_IVT_RH700': ['IVT', 'RH700'],
    'pair_T500_T850': ['T500', 'T850'],
    'pair_RH700_W500': ['RH700', 'W500'],
    'pair_IVT_W500': ['IVT', 'W500'],
    'pair_T500_W500': ['T500', 'W500'],
    # --- Three Variable Models (4 total from your script) ---
    'triplet_IVT_T500_RH700': ['IVT', 'T500', 'RH700'],
    'triplet_T500_T850_W500': ['T500', 'T850', 'W500'],
    'triplet_IVT_RH700_W500': ['IVT', 'RH700', 'W500'],
    'triplet_IVT_T500_W500': ['IVT', 'T500', 'W500'],
    # --- Four Variable Models ("Leave-One-Out") (5 total) ---
    'remove_IVT': [v for v in ALL_VARIABLES if v != 'IVT'],
    'remove_T500': [v for v in ALL_VARIABLES if v != 'T500'],
    'remove_T850': [v for v in ALL_VARIABLES if v != 'T850'],
    'remove_RH700': [v for v in ALL_VARIABLES if v != 'RH700'],
    'remove_W500': [v for v in ALL_VARIABLES if v != 'W500'],
    # --- Five Variable Model (1 total) ---
    'all_variables': ALL_VARIABLES,
}

# --- 3. FLEXIBLE DATASET & MODEL ---
class FlexibleAblationDataset(Dataset):
    def __init__(self, data_dir: Path, split: str, variable_indices: list):
        self.split_dir = data_dir / split
        self.predictor_files = sorted(list(self.split_dir.glob('*_predictor.npy')))
        self.stats = joblib.load(data_dir / 'normalization_stats_multi_variable.joblib')
        self.variable_indices = variable_indices

    def __len__(self): return len(self.predictor_files)
    def __getitem__(self, idx):
        pred_path = self.predictor_files[idx]
        targ_path = Path(str(pred_path).replace('_predictor.npy', '_target.npy'))
        full_predictor = np.load(pred_path).astype(np.float32)
        target_data = np.load(targ_path).astype(np.float32)
        predictor_subset = full_predictor[self.variable_indices, :, :]
        mean_subset = self.stats['predictor_mean'][self.variable_indices, None, None]
        std_subset = self.stats['predictor_std'][self.variable_indices, None, None]
        predictor_norm = (predictor_subset - mean_subset) / (std_subset + 1e-8)
        target_norm = (target_data - self.stats['target_mean']) / (self.stats['target_std'] + 1e-8)
        return torch.from_numpy(predictor_norm), torch.from_numpy(target_norm).unsqueeze(0)

class SimplifiedAttention(nn.Module):
    def __init__(self, channels):
        super().__init__()
        self.channel_att = nn.Sequential(nn.AdaptiveAvgPool2d(1), nn.Conv2d(channels, channels // 16, 1), nn.ReLU(inplace=True), nn.Conv2d(channels // 16, channels, 1), nn.Sigmoid())
        self.spatial_att = nn.Sequential(nn.Conv2d(channels, 1, 7, padding=3), nn.Sigmoid())
    def forward(self, x):
        ch_att = self.channel_att(x); x = x * ch_att; sp_att = self.spatial_att(x); x = x * sp_att
        return x

class FlexibleAblationModel(nn.Module):
    def __init__(self, input_channels, base_channels=64, depth=4, use_attention=True):
        super().__init__()
        self.use_attention, self.depth = use_attention, depth
        self.channels = [base_channels * min(2**i, 8) for i in range(depth)]
        self.encoders, self.downsamplers = nn.ModuleList(), nn.ModuleList()
        in_ch = input_channels
        for i, out_ch in enumerate(self.channels):
            self.encoders.append(self._conv_block(in_ch, out_ch))
            if i < len(self.channels) - 1: self.downsamplers.append(nn.Conv2d(out_ch, out_ch, 3, stride=2, padding=1))
            in_ch = out_ch
        bottleneck_ch = self.channels[-1]
        self.bottleneck = self._conv_block(self.channels[-1], bottleneck_ch)
        if self.use_attention: self.attention = SimplifiedAttention(bottleneck_ch)
        self.upsamplers, self.decoders = nn.ModuleList(), nn.ModuleList()
        for i in range(depth - 1, -1, -1):
            in_ch = bottleneck_ch if i == depth - 1 else self.channels[i+1]
            out_ch = self.channels[i]
            self.upsamplers.append(nn.ConvTranspose2d(in_ch, out_ch, 2, stride=2))
            self.decoders.append(self._conv_block(out_ch * 2, out_ch))
        self.final_conv = nn.Sequential(nn.Conv2d(self.channels[0], self.channels[0] // 2, 3, padding=1), nn.BatchNorm2d(self.channels[0] // 2), nn.ReLU(inplace=True), nn.Conv2d(self.channels[0] // 2, 1, 1))

    def _conv_block(self, in_ch, out_ch): return nn.Sequential(nn.Conv2d(in_ch, out_ch, 3, padding=1), nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True), nn.Conv2d(out_ch, out_ch, 3, padding=1), nn.BatchNorm2d(out_ch), nn.ReLU(inplace=True))

    def forward(self, x):
        skips = []
        for i in range(len(self.encoders)):
            x = self.encoders[i](x); skips.append(x)
            if i < len(self.downsamplers): x = self.downsamplers[i](x)
        x = self.bottleneck(x)
        if self.use_attention: x = self.attention(x)
        for i, (up, dec) in enumerate(zip(self.upsamplers, self.decoders)):
            x = up(x); skip = skips[len(skips) - 1 - i]
            if x.shape[-2:] != skip.shape[-2:]: x = F.interpolate(x, size=skip.shape[-2:], mode='bilinear', align_corners=False)
            x = torch.cat([x, skip], dim=1); x = dec(x)
        return self.final_conv(x)

# --- 4. ADVANCED LOSS FUNCTION (for consistency) ---
class AdvancedLoss(nn.Module):
    def __init__(self, mse_weight=0.6, ssim_weight=0.2, gradient_weight=0.2):
        super().__init__()
        self.mse_weight = mse_weight
        self.ssim_weight = ssim_weight
        self.gradient_weight = gradient_weight
        self.mse_loss = nn.MSELoss()
        self.l1_loss = nn.L1Loss()

    def ssim_loss(self, pred, target, window_size=11):
        mu1 = F.avg_pool2d(pred, window_size, 1, padding=window_size//2)
        mu2 = F.avg_pool2d(target, window_size, 1, padding=window_size//2)
        mu1_sq, mu2_sq, mu1_mu2 = mu1**2, mu2**2, mu1 * mu2
        sigma1_sq = F.avg_pool2d(pred**2, window_size, 1, padding=window_size//2) - mu1_sq
        sigma2_sq = F.avg_pool2d(target**2, window_size, 1, padding=window_size//2) - mu2_sq
        sigma12 = F.avg_pool2d(pred*target, window_size, 1, padding=window_size//2) - mu1_mu2
        c1, c2 = 0.01**2, 0.03**2
        ssim_map = ((2*mu1_mu2 + c1) * (2*sigma12 + c2)) / ((mu1_sq + mu2_sq + c1) * (sigma1_sq + sigma2_sq + c2))
        return 1 - ssim_map.mean()

    def gradient_loss(self, pred, target):
        sobel_x = torch.tensor([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=torch.float32).view(1, 1, 3, 3).to(pred.device)
        sobel_y = torch.tensor([[-1, -2, -1], [0, 0, 0], [1, 2, 1]], dtype=torch.float32).view(1, 1, 3, 3).to(pred.device)
        pred_grad_x = F.conv2d(pred, sobel_x, padding=1)
        pred_grad_y = F.conv2d(pred, sobel_y, padding=1)
        target_grad_x = F.conv2d(target, sobel_x, padding=1)
        target_grad_y = F.conv2d(target, sobel_y, padding=1)
        return self.l1_loss(pred_grad_x, target_grad_x) + self.l1_loss(pred_grad_y, target_grad_y)

    def forward(self, pred, target):
        mse = self.mse_loss(pred, target)
        ssim = self.ssim_loss(pred, target)
        grad = self.gradient_loss(pred, target)
        return (self.mse_weight * mse + self.ssim_weight * ssim + self.gradient_weight * grad)

# --- 5. CONSISTENT TRAINING FUNCTION ---
def train_one_model(config):
    model_name = config['name']
    variable_names = config['variables']
    variable_indices = [ALL_VARIABLES.index(v) for v in variable_names]
    num_channels = len(variable_indices)

    print("\n" + "="*80)
    print(f"🔬 STARTING ABLATION RUN: {model_name}")
    print(f"   Variables ({num_channels}): {', '.join(variable_names)}")
    print("="*80)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    train_dataset = FlexibleAblationDataset(DATA_DIR, 'train', variable_indices)
    val_dataset = FlexibleAblationDataset(DATA_DIR, 'val', variable_indices)
    train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=2, pin_memory=True)
    val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=2, pin_memory=True)

    model = FlexibleAblationModel(input_channels=num_channels).to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=LEARNING_RATE)
    loss_fn = AdvancedLoss() # Using the consistent, advanced loss function
    scaler = torch.cuda.amp.GradScaler(enabled=torch.cuda.is_available())

    best_val_loss = float('inf')
    patience_counter = 0

    for epoch in range(EPOCHS):
        model.train()
        pbar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{EPOCHS} [Train]")
        for predictor, target in pbar:
            predictor, target = predictor.to(device), target.to(device)
            optimizer.zero_grad()
            with torch.cuda.amp.autocast(enabled=torch.cuda.is_available()):
                prediction = model(predictor)
                loss = loss_fn(prediction, target)
            scaler.scale(loss).backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), GRADIENT_CLIP)
            scaler.step(optimizer)
            scaler.update()
            pbar.set_postfix({'loss': f"{loss.item():.4f}"})

        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for predictor, target in val_loader:
                predictor, target = predictor.to(device), target.to(device)
                prediction = model(predictor)
                val_loss += loss_fn(prediction, target).item()

        avg_val_loss = val_loss / len(val_loader)
        print(f"Epoch {epoch+1} | Val Loss: {avg_val_loss:.6f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            patience_counter = 0
            model_path = OUTPUT_DIR / f"ablation_{model_name}.pt"
            metadata_path = OUTPUT_DIR / f"ablation_{model_name}.json"
            torch.save(model.state_dict(), model_path)
            with open(metadata_path, 'w') as f:
                json.dump(config, f, indent=2)
            print(f"   ✅ Best model saved to {model_path}")
        else:
            patience_counter += 1

        if patience_counter >= EARLY_STOPPING_PATIENCE:
            print(f"   🛑 Early stopping triggered.")
            break

    print(f"🔬 FINISHED ABLATION RUN: {model_name} | Best Val Loss: {best_val_loss:.6f}")
    gc.collect()
    torch.cuda.empty_cache()

# --- 6. MAIN EXECUTION SCRIPT ---
def main():
    for name, variables in ABLATION_CONFIGS.items():
        config = {"name": name, "variables": variables}
        train_one_model(config)

    print("\n\n🎉🎉🎉 All ablation study models trained successfully! 🎉🎉🎉")

if __name__ == "__main__":
    main()



🔬 STARTING ABLATION RUN: single_IVT
   Variables (1): IVT


Epoch 1/25 [Train]: 100%|██████████| 150/150 [00:49<00:00,  3.02it/s, loss=0.7096]


Epoch 1 | Val Loss: 0.710103
   ✅ Best model saved to /content/drive/My Drive/AR_Downscaling/ablation_study_models/ablation_single_IVT.pt


Epoch 2/25 [Train]: 100%|██████████| 150/150 [00:33<00:00,  4.44it/s, loss=0.5540]


Epoch 2 | Val Loss: 0.697578
   ✅ Best model saved to /content/drive/My Drive/AR_Downscaling/ablation_study_models/ablation_single_IVT.pt


...

Epoch 6 | Val Loss: 4.105523


Epoch 7/25 [Train]: 100%|██████████| 150/150 [00:31<00:00,  4.74it/s, loss=2.4264]


Epoch 7 | Val Loss: 4.712220


Epoch 8/25 [Train]: 100%|██████████| 150/150 [00:31<00:00,  4.73it/s, loss=1.5335]


Epoch 8 | Val Loss: 1.607224
   🛑 Early stopping triggered.
🔬 FINISHED ABLATION RUN: all_variables | Best Val Loss: 0.725073


🎉🎉🎉 All ablation study models trained successfully! 🎉🎉🎉
